## Unmasking Database Vulnerabilities: Zero-Knowledge Schema Inference Attacks in Text-to-SQL Systems

In [1]:
# IMPORTS
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, AutoModelForCausalLM, pipeline
import torch
import openai
import os

## T5-Large

Model available at: https://huggingface.co/gaussalgo/T5-LM-Large-text2sql-spider

In [3]:
model_path = 'gaussalgo/T5-LM-Large-text2sql-spider'
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

def generate_sql_t5_large(question, schema):
    input_text = f"Question: {question} Schema: {schema}"
    model_inputs = tokenizer(input_text, return_tensors="pt")
    outputs = model.generate(**model_inputs, max_new_tokens=256)
    output_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    
    sql_query = output_text[0]
    return sql_query

## SQLCoder

Model available at: https://huggingface.co/defog/sqlcoder-7b-2

In [2]:
model_name = 'defog/sqlcoder-7b-2'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True, device_map='auto')
model.to(device)

def generate_sql_sqlcoder(question, schema):
    input_text = f"-- Given the following SQL table definitions, answer the question by writing a SQL query.\n\n{schema}\n\n-- {question}\nSELECT"
    inputs = tokenizer.encode(input_text, return_tensors='pt').to(device)

    outputs = model.generate(
        inputs,
        max_new_tokens=256,
        temperature=0.1,
        top_p=0.9,
        do_sample=False,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
    )

    output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    sql_query = output_text.split('\n')[-1]

    if not sql_query.strip().upper().startswith('SELECT'):
        sql_query = 'SELECT' + sql_query

    return sql_query.strip()

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

## Code-Llama2

Model available at: https://huggingface.co/support-pvelocity/Code-Llama-2-7B-instruct-text2sql

In [4]:
model_name = 'support-pvelocity/Code-Llama-2-7B-instruct-text2sql'
tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map='auto',
    load_in_4bit=True,
    torch_dtype=torch.float16,
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
text_gen = pipeline(
    'text-generation',
    model=model,
    tokenizer=tokenizer,
    device=device.index if device.type == 'cuda' else -1,
)

def generate_sql_codellama(question, schema):
    prompt = f"[INST] Write SQLite query to answer the following question given the database schema. Please wrap your code answer using ```:\nSchema: {schema}\nQuestion: {question}\n[/INST]\n"

    generated_text = text_gen(
        prompt,
        max_new_tokens=256,  
        temperature=0.1,     
        top_p=0.9,           
        do_sample=False,     
        pad_token_id=tokenizer.eos_token_id,
    )[0]['generated_text']


    if '```' in generated_text:
        sql_query_parts = generated_text.split('```')
        if len(sql_query_parts) >= 3:
            sql_query = sql_query_parts[2].strip()
        else:
            sql_query = sql_query_parts[-1].strip()
    else:
        sql_query = generated_text

    return sql_query.strip()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## GPT-4

In [6]:
openai.api_key = os.environ['OPENAI_API_KEY']

def generate_sql_gpt4(question, schema):
    system_prompt = (
        "You are an expert SQL assistant. Given a database schema and a user's question, "
        "generate the SQL query that answers the question. Provide only the SQL query without any additional text."
    )

    user_message = f"""
Database Schema:
{schema}

Question:
{question}

SQL Query:
"""
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_message}
    ]

    response = openai.ChatCompletion.create(
        model="gpt-4-0125-preview",
        messages=messages,
        temperature=0,
        max_tokens=500,
    )

    sql_query = response['choices'][0]['message']['content'].strip()

    return sql_query

## LLAMA2

In [5]:
model_path = 'path_to_your_llama2_model'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.float16,
    device_map='auto',
)
model.to(device)

def generate_sql_llama2(question, schema):
    system_prompt = "You are an AI assistant that generates SQL queries based on a database schema and a user's question. Provide only the SQL query without any additional text."

    user_input = f"Given the following database schema:\n{schema}\n\nWrite an SQL query to answer the following question:\n{question}"

    prompt = f"<s>[INST] <<SYS>>\n{system_prompt}\n<</SYS>>\n\n{user_input}\n[/INST]"

    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    outputs = model.generate(
        **inputs,
        max_new_tokens=256,
        do_sample=False,     
        pad_token_id=tokenizer.eos_token_id,
    )

    output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    assistant_reply = output_text.split('[/INST]')[-1].strip()

    if '```' in assistant_reply:
        sql_query = assistant_reply.split('```')[1].strip()
    else:
        sql_query = assistant_reply.strip()
        if '\n' in sql_query:
            sql_query = '\n'.join(sql_query.split('\n')[1:]).strip()

    return sql_query

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## LLAMA3

In [14]:
model_path = 'path_to_your_llama3_model'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.float16,
    device_map='auto',
)
model.to(device)


def generate_sql_llama3(question, schema):
    """
    Parameters:
    - question (str): The natural language question.
    - schema (str): The database schema in SQL CREATE TABLE format.

    Returns:
    - str: The generated SQL query.
    """

    prompt = (
        "You are an AI assistant that generates SQL queries based on a database schema and a user's question.\n"
        "Provide only the SQL query without any additional text, explanations, or comments.\n\n"
        f"Database Schema:\n{schema}\n\n"
        f"Question:\n{question}\n\n"
        "SQL Query:"
    )

    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    outputs = model.generate(
        **inputs,
        max_new_tokens=256,
        do_sample=False,     # Use greedy decoding for deterministic output
        pad_token_id=tokenizer.eos_token_id,
    )

    output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    sql_query = output_text[len(prompt):].strip()
    sql_query = sql_query.split('\n')[0].strip()

    return sql_query

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]